!pip install -r requirements.txt

In [1]:
import os
import time
import requests

from dotenv import load_dotenv
from selenium import webdriver
from bs4 import BeautifulSoup
from mistralai import Mistral

load_dotenv()

True

In [2]:
# Environment variables
EMAIL_MONDE = os.getenv("EMAIL_MONDE")
PASSWORD_MONDE = os.getenv("PASSWORD_MONDE")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
MODEL = "mistral-large-latest"

# URLs
URL_ARTICLE = "https://www.lemonde.fr/idees/article/2025/09/12/comment-la-politique-de-l-offre-detourne-l-argent-public-au-profit-des-plus-riches_6640595_3232.html"
URL_ARTICLE = "https://www.lemonde.fr/idees/article/2025/09/14/patrick-artus-aux-etats-unis-le-capitalisme-de-monopole-provoque-sous-investissement-et-concentration-de-la-richesse_6641078_3232.html"
URL_COMMENTAIRES = URL_ARTICLE + "?contributions&page="

In [3]:
# Create a new instance of the Chrome driver
driver = webdriver.Firefox()

# Open Google
#driver.get("https://www.lemonde.fr/")
driver.get("https://secure.lemonde.fr/sfuser/connexion")

input_element = driver.find_element("id", "email")
input_element.send_keys(EMAIL_MONDE)

password_input = driver.find_element("id", "password")
password_input.send_keys(PASSWORD_MONDE)

validate_button = driver.find_element("id", "submit-button")
validate_button.click()

all_comments = []

next_page = True
page = 0

while next_page:

    page += 1
    print(f"Loading page {page}...")
    driver.get(URL_COMMENTAIRES + str(page))
    #time.sleep(2)  # Wait for 2 seconds to allow the page to load

    html = driver.page_source

    soup = BeautifulSoup(html)
    #soup = BeautifulSoup(html, 'html.parser')

    # Find all elements with class="comment_content"
    comment_contents = soup.find_all(class_="comment__content")

    if len(comment_contents) == 0:
        next_page = False
        print("No more comments found.")
        break

    comment_list = [comment.text for comment in comment_contents]

    all_comments.extend(comment_list)

# Close the browser
driver.quit()

Loading page 1...
Loading page 2...
Loading page 3...
Loading page 4...
No more comments found.


In [4]:
len(all_comments)

51

In [5]:
len(comment_contents)

0

In [6]:
comment_txt = "\n".join(all_comments)

In [7]:
PROMPT_ANALYSE = f"""Tu es un expert en analyse de texte. 
Tu vas analyser les commentaires suivants issus d'un article du journal Le Monde. Chaque commentaire est séparé par un saut de ligne.
Voici les commentaires à analyser :{comment_txt}"""

In [8]:
PROMPT_ANALYSE = f"""Tu es un expert en analyse de texte. 
Tu vas analyser les commentaires suivants issus d'un article du journal Le Monde. Chaque commentaire est séparé par un saut de ligne.
Donne moi la liste des thèmes abordés dans ces commentaires, ainsi que le nombre d'occurrences pour chaque thème.
Voici les commentaires à analyser :{comment_txt}"""

In [9]:
client = Mistral(api_key=MISTRAL_API_KEY)

chat_response = client.chat.complete(
    model= MODEL,
    messages = [
        {
            "role": "user",
            "content": PROMPT_ANALYSE,
        },
    ]
)
print(chat_response.choices[0].message.content)

Voici une analyse thématique des commentaires, avec le nombre d'occurrences pour chaque thème principal. Certains thèmes se recoupent ou sont abordés sous des angles différents, mais j’ai regroupé les idées centrales.

---

### **Liste des thèmes et occurrences**

1. **Capitalisme et monopoles** (32 occurrences)
   - Critique du capitalisme de monopole (ex. : GAFAM, banques, télécoms, etc.).
   - Comparaison avec les monopoles historiques (ex. : "barons voleurs" aux États-Unis, monopoles d'État en France).
   - Débat sur la régulation (ou son absence) des monopoles.
   - Exemples : *"Le capitalisme de monopole n'est jamais que la suite du néolibéralisme"*, *"Les monopoles publics français (SNCF, EDF) vs. monopoles privés américains"*, *"La finance est un monopole de fait"*.

2. **Néolibéralisme et dérégulation** (20 occurrences)
   - Critique du néolibéralisme comme cause des inégalités et des monopoles.
   - Références à Reagan, Thatcher, et leur héritage.
   - Débat sur la définition

In [10]:
PROMPT_BOT = f"""Tu es un expert détection d'ingérence par des bot russes.
Parmi tous les commentaires suivantes, séparés par des retours à la ligne, indique celui qui est le plus à même d'être issu d'un bot russe :{comment_txt}"""

chat_response = client.chat.complete(
    model= MODEL,
    messages = [
        {
            "role": "user",
            "content": PROMPT_BOT,
        },
    ]
)
print(chat_response.choices[0].message.content)